# Зависимости

In [8]:
%tensorflow_version 2.x
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

from mycsv import split_file
from dataset import load_dataset
from transform import Converter
from shortcuts import lzip

# Загрузка модели

In [ ]:
MODEL_PATH = 'fnp_lstm_maxlen250_e25_p722.h5'
model = load_model(MODEL_PATH)
model.summary()

# Загрузка данных для исследования

In [3]:
split_file('dataset.csv', split_ratio=0.8)

После разделения датасета в дериктории появиться два файла (тренировочный сет и тестовый).

* `maxlen` задает максимальную длину последовательности узлов функции (средняя длина последовательности ~50)

* `verbose` задает многословность функций обработчиков (то столько будет выводится информации во время обратоки); доступные значения от 0 до 3

In [3]:
train_file_name = 'train_dataset.csv'
test_file_name = 'test_dataset.csv'
maxlen = 250
verbose = 1

Загрузка конвертеров из файлов.

In [4]:
data_converter = Converter.from_file('dictionary_8241.json')
marks_converter = Converter.from_file('dictionary_112.json')

Загрузка данных для исследования.

In [ ]:
dataset = load_dataset(train_file_name, 
                       test_file_name, 
                       maxlen,
                       data_converter, 
                       marks_converter,
                       verbose
                       )
x_train, y_train, x_test, y_test, data_converter, marks_converter = dataset

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Доля верных ответов на тестовых данных составляет:',round(scores[1]* 100, 4),'%')

Соединение датасетов и добавление строковых лейбелов

In [9]:
# Создание списка лейбелов в соотвествии с OHE
labels = []

for y in y_train:
  labels.append(marks_converter.index_to_item(np.where(y == 1)[0][0]))
train = list(lzip(x_train, y_train, labels))
labels.clear()

for y in y_test:
  labels.append(marks_converter.index_to_item(np.where(y == 1)[0][0]))
test = list(lzip(x_test, y_test, labels))

# Исследование работы модели на исходном датасете

На данном этапе следует проанализировать работу модели на примерах тестовой выборки, с целью исключения функций, которые приводят к коллизиям.

Для этого можно проанализировать следующие параметры:

1. Вероятность правильного определения функции для каждого имени.

2. Самая частотная ошибка при неправильном предсказании.

### Вероятность правильного определения функции для каждого имени.

Функция для предсказания одного примера

In [11]:
def make_prediction(model, sample):
  array = np.array(sample)
  array = (np.expand_dims (array, 0))
  return model.predict(array)[0]

Общее количсество функций для каждого имени.

In [12]:
def count_func_names(data):
  counter_names = {}
  for sample, ohe, name in data:
    counter_names[name] = counter_names.get(name, 0) + 1
  return counter_names

**Добавление к данным значений предсказания модели.**

Предсказание представляет из себя массив вероятностей. Каждый элемент массива стоит в соответствии с элементом one hot encoding (ohe) метки.

Пример:

ohe: `[0, 0, 1, 0...] # единица - метка` 

predict: `[0.001, 0.003, 0.8, 0.00001...] # предсказывает метку с вероятностью 80 %`

In [ ]:
# TODO: добавить предсказания к данным

Сумма вероятностей для каждого имени функции (оптимизировать).

In [14]:
counter_prob = {}
for i, x in enumerate(x_test):
  predict = make_prediction(model, x)
  index = np.argmax(y_test[i])
  label = marks_converter.index_to_item(index)
  counter_prob[label] = counter_prob.get(label, 0) + predict[index]

Сумма всех вероятностей для одного имени деленное на количество функций в выборке дает среднюю вероятность правильного определения функции.
$$
counter\_prob_i / counter\_names_i = result_i
$$

In [15]:
result = {}
for key, score in counter_prob.items():
  result[key] = score / counter_names[key]

In [17]:
result = [(key, value) for key, value in result.items()]
result.sort(key=lambda x: x[1])

In [ ]:
result

# Исследование работы модели на реальных примерах

**Загрузка реальных сниппетов функций**

In [ ]:
SNIPPETS_PATH = 'real_snippets.csv'
snippets = load_dataset(SNIPPETS_PATH, 
                       SNIPPETS_PATH, 
                       maxlen,
                       data_converter, 
                       marks_converter,
                       verbose
                       )
x_snippets, y_snippets, x_test, y_test, data_converter, marks_converter = snippets